In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import tensorflow_hub as hub
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np

/Users/raphaelzacharias/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/raphaelzacharias/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/raphaelzacharias/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/rap

In [2]:
# Create an image generator
##########################
import os
image_generator = ImageDataGenerator(rescale=1/255)

# A function that returns X_train,X_valid,...
#############################################
def get_data(target_size):
    ## The training set
    trainset = image_generator.flow_from_directory(
        os.path.join('swissroads', 'train'),batch_size=280, target_size=(target_size,
        target_size),shuffle=False)
    X_train,y_train = trainset.next()
    files_names_train = trainset.filenames
    ## The valid set
    validset = image_generator.flow_from_directory(
        os.path.join('swissroads', 'valid'), batch_size=139,target_size=(target_size,
        target_size),shuffle=False)
    X_valid,y_valid = validset.next()
    files_names_valid = validset.filenames
    ## The test set
    testset = image_generator.flow_from_directory(
        os.path.join('swissroads', 'test'), batch_size=50,target_size=(target_size,
        target_size),shuffle=False)
    X_test,y_test = testset.next()
    files_names_test = testset.filenames
    ## Label's name
    names = np.array(list(trainset.class_indices.keys()))
    ## Return values
    return X_train,X_valid,X_test,y_train,y_valid,y_test,files_names_train,files_names_valid,files_names_test,names 

In [3]:
## Original images
##################
X_train,X_valid,X_test,y_train,y_valid,y_test,files_names_train,files_names_valid,files_names_test,names  = get_data(32)

Found 280 images belonging to 6 classes.
Found 139 images belonging to 6 classes.
Found 50 images belonging to 6 classes.


In [4]:
## Images from which we extract high level features
###################################################
O_train,O_valid,O_test,_,_,_,_,_,_,_ = get_data(224)

Found 280 images belonging to 6 classes.
Found 139 images belonging to 6 classes.
Found 50 images belonging to 6 classes.


In [5]:
## Shuffle train and valid
np.random.seed(0)
idx_train = np.arange(len(y_train))
idx_valid = np.arange(len(y_valid))
np.random.shuffle(idx_train)
np.random.shuffle(idx_valid)

## train
X_train = X_train[idx_train]
y_train = y_train[idx_train]
files_names_train = np.array(files_names_train)
files_names_train = files_names_train[idx_train]
O_train = O_train[idx_train]

## valid
X_valid = X_valid[idx_valid]
y_valid = y_valid[idx_valid]
files_names_valid = np.array(files_names_valid)
files_names_valid = files_names_valid[idx_valid]
O_valid = O_valid[idx_valid]

## Change the shape of y
y_train = np.argmax(y_train,axis=1)
y_valid = np.argmax(y_valid,axis=1)
y_test = np.argmax(y_test,axis=1)

In [6]:
# Create graph
img_graph = tf.Graph()

with img_graph.as_default():
    # Download module
    module_url = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2'
    feature_extractor = hub.Module(module_url)

    # Create input placeholder
    input_imgs = tf.placeholder(dtype=tf.float32, shape=[None, 224, 224, 3])

    # A node with the features
    imgs_features = feature_extractor(input_imgs)

    # Collect initializers
    init_op = tf.group([
        tf.global_variables_initializer(), tf.tables_initializer()
    ])

img_graph.finalize()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
# Create a session
sess = tf.Session(graph=img_graph)

# Initialize it
sess.run(init_op)

# Extract features
features_train = sess.run(imgs_features, feed_dict={input_imgs: O_train})
features_valid = sess.run(imgs_features, feed_dict={input_imgs: O_valid})
features_test = sess.run(imgs_features, feed_dict={input_imgs: O_test})

print('train : ',features_train.shape)
print('valid : ',features_valid.shape)
print('test : ',features_test.shape)

train :  (280, 1280)
valid :  (139, 1280)
test :  (50, 1280)


In [8]:
## Create npz files
###################
np.savez('data_train.npz', features=X_train, targets=y_train, highfeat=features_train,
        names = names,filenames=files_names_train)
np.savez('data_valid.npz', features=X_valid, targets=y_valid, highfeat=features_valid,
        names =names,filenames=files_names_valid)
np.savez('data_test.npz', features=X_test,targets=y_test, highfeat=features_test,
        names=names,filenames=files_names_test)